In [1]:
# Import libraries
import os
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

In [2]:
# Edit this according to your home directory and where you run this code
home_path = '/Users/skylargale/Documents/VSCode/'

In [3]:
# Step 1: Keep only the relevant data

# Define the path where the clean data is stored
clean_data_dir = os.path.join(home_path, 'MLGEO2024_SeaIcePrediction/data/clean/')

# Load the clean data
data = xr.open_dataset(clean_data_dir + 'clean_nh_monthly_sic_19782024.nc')

In [ ]:
# Step 1: Keep only the relevant data

# Land mask the data so only values between 0 and 1 are kept (all others become nan)
combined_masked = data.where((data['cdr_seaice_conc_monthly'] >= 0) & (data['cdr_seaice_conc_monthly'] <= 1))

# Get data only north of 50N
combined_north = combined_masked.where(data['lat'] > 50, drop=True)

# Regrid the lat x lon to be lon=np.linspace(-180, 180, 1440) and lat=np.linspace(50, 90, 161)
combined_regridded = combined_north.interp(lat=np.linspace(50, 90, 161), lon=np.linspace(-180, 180, 1440))

# Rename the 'cdr_seaice_conc_monthly' variable for ease of use
combined_cleaned = combined_regridded.rename({'cdr_seaice_conc_monthly': 'monthly_sic'})

# Keep only September data
september_data = combined_cleaned.where(combined_cleaned['time.month'] == 9, drop=True)

In [14]:
# Step 2: Save the combined dataset to a new .nc file for ai-ready data
output_path = os.join.path(home_path, 'MLGEO2024_SeaIcePrediction/data/ai_ready')
output_filename = 'ready_nh_sept_sic_19792023.nc'
september_data.to_netcdf(output_filename)
print(f'Saved combined dataset to {os.join.path(output_path, output_filename)}')

In [ ]:
# Step 7: Save the combined dataset to a new .nc file for ai-ready data
output_filename = 'NH_Sept_SIC_19792023.nc'
september_data.to_netcdf(output_filename)
print(f'Saved combined dataset to {output_filename}')

In [ ]:
# Step 8: Check output of combined data file
ds_test = xr.open_dataset('NH_Sept_SIC_19792023.nc')
ds_test

In [ ]:
# Now we can analyze the data in this xarray dataset

# For example, we can calculate the mean sea ice concentration over time
# sic = ds_test['monthly_sic'].mean(dim='time')

# Or select the first month/specific month and year
sic = ds_test['monthly_sic'].isel(time=-1)

# Or Find the difference between the first and last Septembers
# sic = ds_test['monthly_sic'].isel(time=-1) - ds_test['monthly_sic'].isel(time=0)

# Get the land mask
sic_masked = sic.where(sic <= 1.0)

# Mask the NaN values
sic_masked = np.ma.masked_invalid(sic_masked)

# Define a color map
cmap = plt.cm.get_cmap('Blues')
cmap.set_bad(color='lightgrey')

# Plot the sea ice concentration
plt.figure(figsize=(8, 6))

# Add the sea ice concentration
# plt.contourf(sic_masked, levels=np.linspace(0, 1, 11), cmap=cmap, origin='upper')
plt.imshow(sic_masked, cmap=cmap, interpolation='none', aspect='auto')

# Add a colorbar
plt.colorbar(label='Grid Cell Concentration')

# Remove x and y labels
plt.gca().set_xticks([])
plt.gca().set_yticks([])

# Add a title
plt.title('Sea Ice Concentration')

# Show the plot
plt.show()

In [ ]:
# We can also calculate the annual mean sea ice concentration

# Want the mean sea ice concentration where the concentration is less than or equal to 1.0
sic = np.where(sic <= 1.0, sic, np.nan)

# Reshape the first dimension
sic = sic.reshape(-1, 1440, 161)

# Take the mean of the sea ice concentration each year
annual_mean_sic = np.nanmean(sic, axis=(1, 2))

# Plot the annual mean sea ice concentration
plt.figure(figsize=(10, 6))

# Plot the annual mean sea ice concentration
plt.plot(annual_mean_sic)

# Add a linear trend
trend = np.polyfit(range(len(annual_mean_sic)), annual_mean_sic, 1)
plt.plot(np.polyval(trend, range(len(annual_mean_sic))), 'r--')

# Write the trend in the lower left corner of the plot
plt.text(1, 0.01, f'y = {trend[0]:.5f}x + {trend[1]:.5f}', color='red', fontsize=14)

# Adjust the x tick labels so they begin at 1978
plt.xticks(range(0, len(annual_mean_sic), 5), range(1979, 2024, 5)[:len(range(0, len(annual_mean_sic), 5))])

# Add a grid
plt.xlim(0, len(annual_mean_sic))
plt.grid()

# Add labels and a title
plt.xlabel('Year')
plt.ylabel('Mean Sea Ice Concentration')
plt.title('September Mean Sea Ice Concentration')

# Show the plot
plt.show()